<a href="https://colab.research.google.com/github/Tonic4to/Disease-Identification-And-prevention/blob/main/Pest_%2B_Disease_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Models:

CNN image classification - Identifies the crop, identifies the parasite.

# CNN Image Classification


In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [5]:
from google.colab import auth
from google.colab import drive
import json
import os

# Authenticate and mount Google Drive.
auth.authenticate_user()
drive.mount('/content/drive')

# Path to the file where the token is stored (e.g., /content/drive/My Drive/colab_secrets.json)
secrets_file = '/content/drive/My Drive/colab_secrets.json'

# Load the token from the secrets file
with open(secrets_file) as f:
    secrets = json.load(f)
    github_token = secrets['GITHUB_TOKEN']

# Install Kaggle package
!pip install kaggle

# Clone the repository using the token
!git clone https://{github_token}@github.com/Tonic4to/Disease-Identification-And-prevention.git

# Run the shell script to set up Kaggle configuration
!bash Disease-Identification-And-prevention/config/setup_kaggle.sh

# Download a dataset using the Kaggle API
!kaggle datasets download -d marquis03/plants-classification

# Create a directory to unzip the dataset
!mkdir -p dataset_directory

# Unzip the dataset into the created directory
!unzip plants-classification.zip -d dataset_directory

# List the contents of the dataset directory to verify
!ls dataset_directory


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'Disease-Identification-And-prevention' already exists and is not an empty directory.
Disease-Identification-And-prevention/config/setup_kaggle.sh: line 2: $'\r': command not found
fatal: destination path 'Disease-Identification-And-prevention' already exists and is not an empty directory.
Disease-Identification-And-prevention/config/setup_kaggle.sh: line 5: $'\r': command not found
Disease-Identification-And-prevention/config/setup_kaggle.sh: line 7: cd: $'Disease-Identification-And-prevention/config\r': No such file or directory
Disease-Identification-And-prevention/config/setup_kaggle.sh: line 8: $'\r': command not found
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json'$'\r': No such file or directory
Disease-Identification-And-prevention/config/setup_kaggle.sh: line 13: $'\r':

In [ ]:
'''
#Depreciated - use bash script method
!git clone https://github.com/Tonic4to/Disease-Identification-And-prevention.git
!pip install kaggle
%cd Disease-Identification-And-prevention/config
#Moves Kaggle.json to API location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#Downloads Dataset
!kaggle datasets download -d marquis03/plants-classification
'''